# Step 1 - Subset & Ressampling & Mask w/ IdePix

In [3]:
import os
import subprocess

# Here we define the input and output folders and the file name pattern for the input and output files.
 
output_folder = 'l1c_ressampled_masked/'
name_start = 'S2_MSIL1C_'
name_end = '_samp_mask.dim'
input_folder = 'L1C_zip_files/'

def process_zip_files(output_folder, name_start, name_end, input_folder):
    """
    Process the zip files in the input folder by resampling and masking them using the gpt tool.
    
    This function iterates over the zip files in the input folder, extracts the acquisition date from the file name,
    and generates the output file path. It then executes the gpt command to subset, resample and mask the input file.

    """

    for zip_file in os.listdir(input_folder):
        if zip_file.endswith('.zip'):
            product_basename = os.path.basename(zip_file)
            ac_date = product_basename.split('_')[2]
            output_pathname = os.path.join(output_folder, name_start + ac_date + name_end)
            input_pathname = os.path.join(input_folder, zip_file)

            cmd = [ 
                'gpt',
                'data/STEP1_subset_resample_mask.xml',
                '-PInput=' + input_pathname,
                '-POutput=' + output_pathname
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)
# Here we call the function to process the zip files in the input folder.
process_zip_files(output_folder, name_start, name_end, input_folder)

STDOUT: Executing processing graph
Initializing Elevation Model
100% done.

100% done.

100% done.

100% done.

100% done.

100% done.
15%30%45%60%....70%..90% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.s2tbx.dataio.s2.ortho.S2OrthoProductReaderPlugIn: Building product reader - EPSG:32631
INFO: org.hsqldb.persist.Logger: dataFileCache open start

STDOUT: Executing processing graph
Initializing Elevation Model
100% done.

100% done.

100% done.

100% done.

100% done.

100% do

# Step 2 - Application of C2RCC w/ Meteo Data

In [4]:
import os
import subprocess

# Here we define the input and output folders and the file name pattern for the input and output files.

output_folder = 'c2rcc/'
met_data = 'data/meteo_data.csv'
param_file = 'data/STEP2_c2rcc_correction_adjusted.xml'
input_folder = 'l1c_ressampled_masked/'

def process_c2rcc_correction(input_folder, output_folder, met_data, param_file):
    """
    Process C2RCC correction on the input files in the specified folder.

    Args:
        input_folder (str): Path to the folder containing the input files.
        output_folder (str): Path to the folder where the output files will be saved.
        met_data (str): Path to the meteorological data file.
        param_file (str): Path to the C2RCC correction parameter file.

    Returns:
        None
    """
    old_end = '.dim'
    new_end = '_C2RCC.dim'

    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('.dim'):
            dim_file_path = os.path.join(input_folder, dim_file)
            name = os.path.basename(dim_file_path)
            idate = name.split('_')[2].split('T')[0]

            """
            We read the meteorological data for the acquisition date of the current file and 
            extract the ozone, pressure and temperature values.
            """

            with open(met_data, 'r') as met_file:
                for line in met_file:
                    if idate in line:
                        parts = line.split(',')
                        temp = parts[1]
                        press = parts[2]
                        ozone = parts[3]
                        ozone = ozone.replace('\n', '')
                        break

            output_name = name.replace(old_end, new_end)
            output_path = os.path.join(output_folder, output_name)

            cmd = [
                'gpt',
                'c2rcc.msi',
                '-SsourceProduct=' + dim_file_path,
                '-p', param_file,
                '-Pozone=' + ozone,
                '-Ppress=' + press,
                '-Ptemperature=' + temp,
                '-t', output_path
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

# Running the function
process_c2rcc_correction(input_folder, output_folder, met_data, param_file)

STDOUT: Executing operator...
10%20%...30%33%36%...48%...61%61%64%67%...79%82%...94%.. done.
Writing...
....10%....20%...30%...42%...54%...64%...76%...88%...100% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO: org.esa.s3tbx.c2rcc.ancillary.AtmosphericAuxdataBuilder: Atmospheric auxdata product can't be used. At least one is not specified. Using constant values for ozone (279.259583) and surface pressure (1002.9475).
INFO: org.esa.s

## Step 2.1 - Application of C2RCC w/ Meteo Data (excluding Ozone data)

In [ ]:
import os
import subprocess

output_folder = 'c2rcc_without_ozone/'
met_data = 'data/meteo_data.csv'
param_file = 'data/STEP2_c2rcc_correction.xml'
input_folder = 'l1c_ressampled_masked/'

def process_c2rcc_correction_without_ozone(input_folder, output_folder, met_data, param_file):

    
    """
    Process C2RCC correction on the input files in the specified folder.

    Args:
        input_folder (str): Path to the folder containing the input files.
        output_folder (str): Path to the folder where the output files will be saved.
        met_data (str): Path to the meteorological data file.
        param_file (str): Path to the C2RCC correction parameter file.

    Returns:
        None
    """

    old_end = '.dim'
    new_end = '_C2RCC.dim'

    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('.dim'):
            dim_file_path = os.path.join(input_folder, dim_file)
            name = os.path.basename(dim_file_path)
            idate = name.split('_')[2].split('T')[0]

            """
            We read the meteorological data for the acquisition date of the current file and 
            extract the pressure and temperature values.
            """

            with open(met_data, 'r') as met_file:
                for line in met_file:
                    if idate in line:
                        parts = line.split(',')
                        temp = parts[1]
                        press = parts[2]
                        break

            output_name = name.replace(old_end, new_end)
            output_path = os.path.join(output_folder, output_name)

            cmd = [
                'gpt',
                'c2rcc.msi',
                '-SsourceProduct=' + dim_file_path,
                '-p', param_file,
                '-Ppress=' + press,
                '-Ptemperature=' + temp,
                '-t', output_path
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

# Running the function
process_c2rcc_correction_without_ozone(input_folder, output_folder, met_data, param_file)

## Step 2.2 - Application of C2RCC w/ Meteo Data + C2-NETS

In [ ]:
import os
import subprocess

output_folder = 'c2rcc_complex_nets/'
met_data = 'data/meteo_data.csv'
param_file = 'data/STEP22_c2rcc_complex_nets_correction.xml'
old_end = '.dim'
new_end = '_C2RCC.dim'

input_folder = 'l1c_ressampled_masked/'

process_c2rcc_correction(input_folder, output_folder, met_data, param_file)


# Step 3 - Bio-optical models calculation

In [1]:
"""
This script performs 6 bands manipulations of Rrs (Remote Sensing Reflectance) to estimate chlorophyll-a concentration.
It applies different equations to calculate the chlorophyll-a concentration using the Rrs values of different bands.

The following equations are used to calculate the chlorophyll-a concentration using the Rrs values of different bands:

1. 3_bands_model_chl_conc - 161.24*((1/ rrs_B4 - 1/ rrs_B5)/(1/ rrs_B6 - 1/ rrs_B5))+28.04
2. MCI - rrs_B5 - rrs_B4 - (rrs_B6 -rrs_B4)*((709-665)/(754-665))
3. empirical_2_bands_equation1 - 136.3 * (rrs_B6/rrs_B4) - 16.2
4. empirical_2_bands_equation2 - 25.28*((rrs_B5/rrs_B4)*(rrs_B5/rrs_B4)) + 14.85 * (rrs_B5/rrs_B4) - 15.18
5. empirical_3_bands_equation1 - 117.42 * ((1/rrs_B4 - 1/rrs_B5))*rrs_B6 + 23.174
6. empirical_3_bands_equation3 - 315.5 * (((1/rrs_B4 - 1/rrs_B5))*rrs_B6)*(((1/rrs_B4 - 1/rrs_B5))*rrs_B6)+ 215.95 * ((1/rrs_B4 - 1/rrs_B5))*rrs_B6 + 25.66

The script takes input files from the 'c2rcc' folder, applies the calculations defined in the 'STEP3_chla_algorithms_calculation.xml' graph file,
and saves the output files with modified names in the 'c2rcc_final_bands' folder.

The script uses the 'gpt' command-line tool to execute the graph file and perform the calculations.

Note: The 'gpt' command-line tool must be installed and accessible in the system's PATH for the script to work properly.
"""

import os
import subprocess

output_folder = 'c2rcc_final_bands/'
input_folder = 'c2rcc/'

graph_file = 'data/STEP3_chla_algorithms_calculation.xml'

old_end = '.dim'
new_end = '_mathbands.dim'

for dim_file in os.listdir(input_folder):
    if dim_file.endswith('.dim'):
        dim_file_path = os.path.join(input_folder, dim_file)
        name = os.path.basename(dim_file_path)

        output_name = name.replace(old_end, new_end)
        output_path = os.path.join(output_folder, output_name)

        cmd = [ 
            'gpt',
            graph_file,
            '-PInput=' + dim_file_path,
            '-POutput=' + output_path
        ]

        result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print("STDOUT:", result.stdout)
        print("STDERR:", result.stderr)


STDOUT: Executing processing graph
.11%.22%.33%.45%.56%.67%.78%...89% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.hsqldb.persist.Logger: dataFileCache open start

STDOUT: Executing processing graph
.11%.22%.33%.45%.56%.67%.78%...89% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.c

# Step 4 - Extract values of the bands

### Results of C2RCC with Ozone Data - window 3x3

In [2]:
import os
import subprocess

output_folder = 'stats_results_windows'
param_file = 'data/STEP4_1_pixel_values_extraction_3x3.xml'

input_folder = 'c2rcc_final_bands/*.dim'

cmd = [
    'gpt',
    param_file,
    '-PInput=' + input_folder,
    '-POutput='+ output_folder
]

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

STDOUT: Executing processing graph
....11%....21%....31%...44%89% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.hsqldb.persist.Logger: dataFileCache open start



### Results of C2RCC with Ozone Data - window 5x5

In [3]:
import os
import subprocess

output_folder = 'stats_results_windows'
param_file = 'data/STEP4_2_pixel_values_extraction_5x5.xml'

input_folder = 'c2rcc_final_bands/*.dim'

cmd = [
    'gpt',
    param_file,
    '-PInput=' + input_folder,
    '-POutput='+ output_folder
]

result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

STDOUT: Executing processing graph
....11%....21%....31%...44%89% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.hsqldb.persist.Logger: dataFileCache open start



### Consolidation of window results

In [4]:
import pandas as pd

# Read the csv files and skipping the first 6 rows 
df_3x3 = pd.read_csv('stats_results_windows/pixEx_3x3__BandMath_measurements.txt', delimiter='\t', skiprows=6)
df_5x5 = pd.read_csv('stats_results_windows/pixEx_5x5_BandMath_measurements.txt', delimiter='\t', skiprows=6)

# Selecting the columns that we want to analyse

columns = ['Date(yyyy-MM-dd)', '3_bands_model_chl_conc_mean', '3_bands_model_chl_conc_sigma',
           '3_bands_model_chl_conc_num_pixels', 'kd_z90max_mean', 'kd_z90max_sigma', 'kd_z90max_num_pixels',
           'conc_tsm_mean', 'conc_tsm_sigma', 'conc_tsm_num_pixels', 'conc_chl_mean', 'conc_chl_sigma',
           'conc_chl_num_pixels', 'MCI_mean', 'MCI_sigma', 'MCI_num_pixels', 'empirical_2_bands_equation1_mean',
           'empirical_2_bands_equation1_sigma', 'empirical_2_bands_equation1_num_pixels',
           'empirical_2_bands_equation2_mean', 'empirical_2_bands_equation2_sigma',
           'empirical_2_bands_equation2_num_pixels', 'empirical_3_bands_equation1_mean',
           'empirical_3_bands_equation1_sigma', 'empirical_3_bands_equation1_num_pixels',
           'empirical_3_bands_equation3_mean', 'empirical_3_bands_equation3_sigma',
           'empirical_3_bands_equation3_num_pixels']

# Creating a new column with the window size of each row
df_3x3 = df_3x3[columns]
df_3x3['window_size'] = 3
df_5x5 = df_5x5[columns]
df_5x5['window_size'] = 5

# Display DataFrame
display(df_3x3)
display(df_5x5)

# Concatenate the two DataFrames and save the results to a csv file
df_windows = pd.concat([df_3x3, df_5x5], ignore_index=True)
display(df_windows)

df_windows.to_csv('data/results_window_calculation_final.csv', index=False)

,Date(yyyy-MM-dd),3_bands_model_chl_conc_mean,3_bands_model_chl_conc_sigma,3_bands_model_chl_conc_num_pixels,kd_z90max_mean,kd_z90max_sigma,kd_z90max_num_pixels,conc_tsm_mean,conc_tsm_sigma,conc_tsm_num_pixels,...,empirical_2_bands_equation2_mean,empirical_2_bands_equation2_sigma,empirical_2_bands_equation2_num_pixels,empirical_3_bands_equation1_mean,empirical_3_bands_equation1_sigma,empirical_3_bands_equation1_num_pixels,empirical_3_bands_equation3_mean,empirical_3_bands_equation3_sigma,empirical_3_bands_equation3_num_pixels,window_size
0,2022-10-07,11.929392,1.234483,9,1.390186,0.224819,9,2.605074,0.719015,9,...,12.878231,1.292255,9,15.350735,0.697266,9,12.682456,1.035649,9,3
1,2022-10-22,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
2,2022-11-01,9.280175,1.255743,9,1.514117,0.253600,9,0.806127,0.053372,9,...,10.148916,1.439241,9,13.844873,0.746850,9,10.505498,1.068507,9,3
3,2022-11-04,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
4,2023-01-18,18.433088,3.422794,9,0.793088,0.095530,9,1.376787,0.551600,9,...,18.316576,2.599415,9,18.699402,1.664171,9,17.945158,2.752066,9,3
5,2023-02-07,13.131081,2.668654,9,1.387007,0.511852,9,2.333530,1.018988,9,...,13.893616,2.983378,9,15.977972,1.562557,9,13.660224,2.294273,9,3
6,2023-02-09,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
7,2023-02-14,6.463346,2.494657,9,2.727365,1.303080,9,0.359492,0.201445,9,...,6.535113,3.694502,9,12.086758,1.653366,9,8.137735,2.251191,9,3
8,2023-02-27,18.099707,1.950006,9,0.892746,0.093844,9,2.064326,0.595679,9,...,18.630674,1.284169,9,18.686720,0.903353,9,17.884700,1.479665,9,3
9,2023-04-03,10.248935,2.193858,9,1.962258,0.666384,9,1.705874,0.690210,9,...,11.016630,2.491593,9,14.365870,1.297220,9,11.270326,1.843018,9,3


,Date(yyyy-MM-dd),3_bands_model_chl_conc_mean,3_bands_model_chl_conc_sigma,3_bands_model_chl_conc_num_pixels,kd_z90max_mean,kd_z90max_sigma,kd_z90max_num_pixels,conc_tsm_mean,conc_tsm_sigma,conc_tsm_num_pixels,...,empirical_2_bands_equation2_mean,empirical_2_bands_equation2_sigma,empirical_2_bands_equation2_num_pixels,empirical_3_bands_equation1_mean,empirical_3_bands_equation1_sigma,empirical_3_bands_equation1_num_pixels,empirical_3_bands_equation3_mean,empirical_3_bands_equation3_sigma,empirical_3_bands_equation3_num_pixels,window_size
0,2022-10-07,12.707679,1.727899,25,1.288658,0.258345,25,2.902083,0.970038,25,...,13.713495,1.733430,25,15.799156,0.964616,25,13.361759,1.458194,25,5
1,2022-10-22,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,5
2,2022-11-01,9.336197,1.914533,25,1.613725,0.586868,25,0.803591,0.255541,25,...,10.193232,2.237619,25,13.873179,1.148377,25,10.563113,1.618603,25,5
3,2022-11-04,12.892994,1.904203,5,1.036004,0.148508,5,0.706938,0.116830,5,...,13.399799,1.872270,5,15.783124,1.050155,5,13.337439,1.581071,5,5
4,2023-01-18,17.615936,3.544894,25,0.832864,0.193146,25,1.404816,0.711962,25,...,17.612406,2.839504,25,18.277273,1.756886,25,17.270805,2.891249,25,5
5,2023-02-07,12.351288,3.011346,25,1.638708,0.982611,25,1.908609,0.974615,25,...,12.863280,3.866551,25,15.491367,1.859570,25,12.957273,2.667486,25,5
6,2023-02-09,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,5
7,2023-02-14,6.525974,3.148817,25,3.207387,2.646154,25,0.414779,0.507206,25,...,6.474755,4.155940,25,12.095517,1.958097,25,8.178027,2.773979,25,5
8,2023-02-27,17.737429,2.225846,25,0.902310,0.181023,25,2.683188,1.484909,25,...,18.136799,1.890491,25,18.451801,1.130966,25,17.513662,1.813983,25,5
9,2023-04-03,9.735119,1.950018,25,2.114878,0.601006,25,1.552345,0.615127,25,...,10.455063,2.303479,25,14.068318,1.168955,25,10.840853,1.668904,25,5


,Date(yyyy-MM-dd),3_bands_model_chl_conc_mean,3_bands_model_chl_conc_sigma,3_bands_model_chl_conc_num_pixels,kd_z90max_mean,kd_z90max_sigma,kd_z90max_num_pixels,conc_tsm_mean,conc_tsm_sigma,conc_tsm_num_pixels,...,empirical_2_bands_equation2_mean,empirical_2_bands_equation2_sigma,empirical_2_bands_equation2_num_pixels,empirical_3_bands_equation1_mean,empirical_3_bands_equation1_sigma,empirical_3_bands_equation1_num_pixels,empirical_3_bands_equation3_mean,empirical_3_bands_equation3_sigma,empirical_3_bands_equation3_num_pixels,window_size
0,2022-10-07,11.929392,1.234483,9,1.390186,0.224819,9,2.605074,0.719015,9,...,12.878231,1.292255,9,15.350735,0.697266,9,12.682456,1.035649,9,3
1,2022-10-22,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
2,2022-11-01,9.280175,1.255743,9,1.514117,0.253600,9,0.806127,0.053372,9,...,10.148916,1.439241,9,13.844873,0.746850,9,10.505498,1.068507,9,3
3,2022-11-04,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
4,2023-01-18,18.433088,3.422794,9,0.793088,0.095530,9,1.376787,0.551600,9,...,18.316576,2.599415,9,18.699402,1.664171,9,17.945158,2.752066,9,3
5,2023-02-07,13.131081,2.668654,9,1.387007,0.511852,9,2.333530,1.018988,9,...,13.893616,2.983378,9,15.977972,1.562557,9,13.660224,2.294273,9,3
6,2023-02-09,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,...,NaN,0.000000,0,NaN,0.000000,0,NaN,0.000000,0,3
7,2023-02-14,6.463346,2.494657,9,2.727365,1.303080,9,0.359492,0.201445,9,...,6.535113,3.694502,9,12.086758,1.653366,9,8.137735,2.251191,9,3
8,2023-02-27,18.099707,1.950006,9,0.892746,0.093844,9,2.064326,0.595679,9,...,18.630674,1.284169,9,18.686720,0.903353,9,17.884700,1.479665,9,3
9,2023-04-03,10.248935,2.193858,9,1.962258,0.666384,9,1.705874,0.690210,9,...,11.016630,2.491593,9,14.365870,1.297220,9,11.270326,1.843018,9,3


### Results of C2RCC with Ozone Data - All parameters

In [5]:
import os
import subprocess

output_folder = 'stats_results_final_bands/'
param_file = 'data/STEP4_3_all_pixels_extraction_StatisticOperator.xml'
input_folder = 'c2rcc_final_bands/'

def process_files(output_folder, param_file, input_folder):
    """
    Process the files in the input folder by running a command line tool that extract 
    the pixel values of bands selected in the parameter file.
    
    The bands selected are: conc_chl, conc_tsm, 3_bands_model_chl_conc, MCI, empirical_2_bands_equation1, 
    empirical_2_bands_equation2, empirical_3_bands_equation1, empirical_3_bands_equation3.

    The output files will be .txt and saved in the output folder.
    """

    old_end = '.dim'
    new_end = '_stats.txt'

    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('.dim'):
            dim_file_path = os.path.join(input_folder, dim_file)
            name = os.path.basename(dim_file_path)

            output_name = name.replace(old_end, new_end)
            output_path = os.path.join(output_folder, output_name)

            cmd = [
                'gpt',
                param_file,
                '-PInput=' + dim_file_path,
                '-POutput='+ output_path
            ]

            result = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("STDOUT:", result.stdout)
            print("STDERR:", result.stderr)

process_files(output_folder, param_file, input_folder)


STDOUT: Executing processing graph
30%60%90% done.

STDERR: INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: GDAL not found on system. Internal GDAL 3.0.0 from distribution will be used. (f1)
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO: org.esa.snap.statistics.ProductLoop:     current product: c:\Users\arthu\Documents\GitHub\remote_sensing_lac_creteil\c2rcc_final_bands\S2_MSIL1C_20221007T104931_samp_mask_C2RCC_mathbands.dim
INFO: org.esa.snap.statistics.StatisticsOp: Successfully computed statistics.

STDOUT: Executing processing graph
30%60%90% done.

STDERR: INFO: org.es

In [6]:
import pandas as pd
import os

def process_data(input_folder):
    """
    Process the data files in the given input folder.
    
    Args:
        input_folder (str): Path to the folder containing the data files.
        
    Returns:
        pd.DataFrame: Consolidated dataframe containing the processed data.
    """
    # Create an empty list to store the dataframes
    dfs = []

    # Iterate over the files in the input folder
    for dim_file in os.listdir(input_folder):
        if dim_file.endswith('_stats.txt'):
            # Extract the date from the file name
            name = os.path.basename(dim_file)
            idate = name.split('_')[2].split('T')[0]

            # Read the data from the file into a dataframe
            df_virtual = pd.read_csv(os.path.join(input_folder, dim_file), sep='\t')
            df_virtual['date'] = pd.to_datetime(idate, format='%Y%m%d')

            # Append the dataframe to the list
            dfs.append(df_virtual)

    # Concatenate all the dataframes into a single dataframe
    consolidated_df = pd.concat(dfs, ignore_index=True)

    return consolidated_df

# Running the function
input_folder = 'stats_results_final_bands/'
consolidated_df = process_data(input_folder)
display(consolidated_df)
consolidated_df.to_csv('data/results_final_bands_final.csv', index=False)


,name,Band,average,max_error,maximum,median,minimum,p80_threshold,p85_threshold,p90_threshold,sigma,total,date
0,world,3_bands_model_chl_conc,11.4282,0.0230,26.2689,12.3727,3.2238,14.2624,14.6772,15.1150,3.7147,794,2022-10-07
1,world,MCI,0.0000,0.0000,0.0010,-0.0000,-0.0000,-0.0000,-0.0000,-0.0000,0.0001,8800,2022-10-07
2,world,conc_chl,10.0543,0.0198,19.7945,10.6735,0.0091,13.0873,14.6107,16.3123,4.9180,794,2022-10-07
3,world,conc_tsm,2.2290,0.0097,9.7472,1.8800,0.0106,3.7592,4.2849,4.7815,1.7885,794,2022-10-07
4,world,empirical_2_bands_equation1,18.7129,0.0384,39.3036,20.9567,0.9211,24.0657,24.5647,25.3707,7.5726,794,2022-10-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,world,empirical_2_bands_equation1,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2023-09-25
239,world,empirical_2_bands_equation2,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2023-09-25
240,world,empirical_3_bands_equation1,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2023-09-25
241,world,empirical_3_bands_equation3,-999.0000,0.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,-999.0000,0,2023-09-25


# NOT USED

### Results of C2RCC without Ozone data

In [ ]:
output_folder = 'stats_results_without_ozone/'
param_file = 'data/[OLD]STEP4_chl_tsm_extraction_all_pixels.xml'
input_folder = 'c2rcc_without_ozone/'

# Call the function to start the processing
process_files(input_folder, output_folder, param_file)

# Save the results to a csv file
consolidated_df = process_data(input_folder)
display(consolidated_df)
consolidated_df.to_csv(input_folder + 'results_without_ozone.csv', index=False)


### Results of Complex Waters Algorithm

In [ ]:
output_folder = 'stats_results_complex_nets/'
param_file = 'data/[OLD]STEP4_chl_tsm_extraction_all_pixels.xml'
input_folder = 'c2rcc_complex_nets/'

# Call the function to start the processing of the statistics extraction files
process_files(input_folder, output_folder, param_file)


# Save the results to a csv file
consolidated_df = process_data(input_folder)
display(consolidated_df)
consolidated_df.to_csv(input_folder + 'results_complex_nets.csv', index=False)
